In [226]:
using MAT, Statistics, LinearAlgebra,Random, Plots, NLsolve, ForwardDiff

Function that reads in all the variables optained that are used in line 93 and onwards and are defined before that.

In [74]:
include("mvnrnd.jl")

mvnrnd (generic function with 1 method)

Turns them into julia variables.

In [75]:
function string_as_varname(s::AbstractString,v::Any)
    s=Symbol(s)
    if size(v,2) == 1 # check if v is a column vector
        @eval (($s) = (vec($v)))
    else
        @eval (($s) = ($v))
    end
end

file = matopen("simulationData.mat")



varnames = names(file) 
#[IO, price, Omega, data, labor, mu, aggtfp, alpha, grossy, stfp, beta, agggdp, Sigma, capital, vadd]
for varname in varnames
    val = read(file, varname)
    # Instantiate the variable with its name in the current scope
    string_as_varname(varname,val)
    end

close(file)

We now want to figure out what every variable stands for, and where it comes from.
So far we know:
- **IO**: Input-Output Matrix
- **STFP** Sectoral TFP Growth per anno
- **$\Omega$**: I think this is the same as in *Definition 3* Es gilt $$(\Omega)_i^j = \frac {(\mathit{IO})_i^j} {\sum_{k=0}^N(\mathit{IO}_k^j)}$$ Also der Anteil eines Sektors $i$ an der Gesamptproduktion des Sektors $j$
- **$\Sigma$**: Covarianz von SFTP
- **$\mu$**: Rowwise mean of stfp. So the mean annual groth of each sector
- **data**: quantity data matrix
- **price**: price data matrix. I think price and data are for our anyalsis quite useless, they are used as building blocks
- **grossy** nominal gross output
- **vadd** nominal value added
- **labor** nominal labor
- **capital** nominal captial
- **alpha**: vadd / grossy.
- **beta**: `beta = (I-diagm(1-alpha)*Omega)' * grossy(:,year-1959)` this is then normalized and filtered. 

This is not used anywhere in the code.

Found way too late, for how often this is used.

In [111]:
#domar_weights = (beta' * inv(I - diagm(1 .-alpha)* Omega))';
domar_weights = (inv(I - diagm(1 .- alpha) * Omega)' * beta);

In [11]:
cum_stfp = cumsum(log.(1 .+ stfp)',dims=1);
cum_stfp_4year = cum_stfp[:,begin:4:end];
covariance_4year = Statistics.cov(cum_stfp_4year,dims=2,corrected=false);

Some elasticities, explaination can be found in section 6

In [137]:
epsilon = ε = .5; 
theta = θ = 0.001; 
sigma = σ =.9; 

Again, not used anymore

In [41]:
trials = 1; #number of draws
#clear Shocks LShocks;
GDP = zeros(trials,1);
variances = (movingvar(stfp',5)'); #rolling estimate of variance of TFP
var_cri = variances[:,22];

var_cri = diag(Sigma)*2; #crisis episode variances



LoadError: UndefVarError: movingvar not defined

L is the steady state allocation of labor
$$L_i = (\beta^T (I - \Omega + \text{diag}(\alpha)\Omega)^{-1})^T \cdot \alpha = (I - (I - \text{diagm}(\alpha)) * \Omega)^{-1}\beta) \cdot \alpha = \lambda \cdot \alpha$$


In [98]:
#L = (beta'*inv(I-diagm(1 .- alpha)*Omega))'.*alpha; #steady-state allocation of labor
L = domar_weights .* alpha;

Here comes the randomness. As far as i can tell the generated vector $x$ has mean $\mu = -\frac 1 2 \text{Var}(\mathit{stfp})$ and $\sigma = \text{Var}(\mathit{stfp})$ Because $\sigma$ is a Diagonal Matrix, there is no correlation between each random number, what makes this rather perculiar. Intuition wise I would say, that this implies, that the shocks are independent of one-another. (As described in  Chapter 6)

$$A_i = e^{x_i}$$

In [101]:
A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));

In [ ]:
$init = e^()$

In [130]:
#init = [exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A));(beta' * inv(I- diagm(1 .- alpha)*Omega))'./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))]  #judicious choice of starting values
x0 = [exp.(-inv(I - diagm(1 .- alpha)*Omega)*log.(A));domar_weights ./exp.(-inv(I - diagm(1 .- alpha) * Omega)*log.(A))];

The `fmincon` function is abused in my opinion. The problem as stated is: $$\min_x 0 \quad \text{s.t} \,\, c(x) = 0$$
However this is simply: Solve $$c(x) = 0$$And for this the `fsolve` function can be utilized.

Lets analyze what $c(x)$ actually is now. (Only `Out` is of importance, the other return values are either unused arguments in the `fmincon` function or the derivatives of `Out`, to make the solver more efficient.



In [45]:
function [outineq, Out, outineq2, OutDeriv] = Simulation_Derivs(X, A, beta, Omega, alpha, epsilon, theta, sigma,L) % no reallocation of labor
    N = length(alpha);
    p = X[1:N];
    y = X[N+1:2*N];

    q = (Omega* (p.^(1-theta))).^(1/(1-theta));
    w = p.*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon)).*(1./L).^(1/epsilon);
    C = w'*L;

    Out(1:N) = p - (diag(A)^(epsilon-1)*(alpha.* w .^ (1-epsilon) + (1-alpha) .*q .^ (1-epsilon))).^(1/(1-epsilon));
    Out(N+1:2*N) = y' -y'*diag(p)^epsilon*diag(A)^(epsilon-1)*diag(q)^(theta-epsilon)*diag(1-alpha)*Omega*diag(p)^(-theta) - beta'*diag(p)^(-sigma)*C;
    Out = Out';
    outineq = [];
    outineq2 = [];

    DQDP = bsxfun(@times, (q.^theta), (p.^(-theta))').*Omega; % 
    DWDP = diag((A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon)).*(1./L).^(1/epsilon)); %checked
    DWDY = (1/epsilon)*diag(p.*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon-1)).*(L).^(-1/epsilon)); %checked
    DCDP = DWDP'*L; %checked
    DCDY = DWDY'*L;%checked
    DOut1DP = eye(N) - diag(diag(A)^(-1)*((alpha.*(w.^(1-epsilon))+(1-alpha).*(q.^(1-epsilon)))).^(epsilon/(1-epsilon)))*...
        (diag(alpha)*diag(w.^(-epsilon))*DWDP+diag(1-alpha)*diag(q.^(-epsilon))*DQDP);

    DOut1DY =  -diag(diag(A.^(-1))*((alpha.*(w.^(1-epsilon))+(1-alpha).*(q.^(1-epsilon)))).^(epsilon/(1-epsilon)))*...
        (diag(alpha)*diag(w.^(-epsilon))*DWDY);

    DOut2DP = -(epsilon * diag(p.^(-theta))*Omega'*diag((p.^(epsilon-1)).*(y).*(q.^(theta-epsilon)).*(1-alpha).*(A.^(epsilon-1)))...
        +(theta-epsilon)*diag(p.^(-theta))*Omega'*diag((p.^(epsilon)).*(y).*(q.^(theta-epsilon-1)).*(1-alpha).*(A.^(epsilon-1)))*DQDP ...
        -sigma*diag(beta.*p.^(-sigma-1))*C+ bsxfun(@times, beta.*(p.^(-sigma)), DCDP')...
        - theta* diag(p.^(-theta-1)).*diag(Omega'*diag((p.^(epsilon)).*(q.^(theta-epsilon)).*(1-alpha).*(A.^(epsilon-1)))*y));

    DOut2DY = eye(N) - (diag(p)^epsilon*diag(A)^(epsilon-1)*diag(q)^(theta-epsilon)*diag(1-alpha)*Omega*diag(p)^(-theta))' - bsxfun(@times, beta.*(p.^(-sigma)), DCDY');
    OutDeriv = [DOut1DP DOut1DY; DOut2DP DOut2DY]';

end

LoadError: syntax: expected "(" in function definition

In [216]:
function problem(X, A, β, Ω, α, ε, θ, σ,L)
    N = length(α)
    p = X[1:N]
    y = X[N+1:end]
    
    Out = zeros(2*N)
    
    q = (Ω * p .^ (1-θ)) .^ (1 / (1 - θ))
    w = p .* (A .^ ((ε - 1)/ε)) .* (α .^ (1 / ε)) .* (y .^ (1/ε)) .* L .^ (-1/ε)
    C = w' * L
  
    Out[1:N] = p - (diagm(A) ^ (ε - 1) * (α .* w .^ (1- ε) + (1 .- α) .* q .^ (1 - ε))) .^ (1/1-ε)
    Out[N+1:end] = y' - y'* diagm(p)^ε * diagm(A)^(ε-1) * diagm(q)^(θ-ε) * diagm(1 .- α) * Ω*diagm(p)^(-θ) - β'*diagm(p)^(-σ)*C
    
    return Out
    end

problem (generic function with 1 method)

In [227]:
problem(x0,A,beta,Omega,alpha,ε,θ,sigma,L)

152-element Vector{Float64}:
 -0.03238866551592301
 -0.07639414868854266
 -0.1093093050886611
  0.050314607704717185
  0.017821267326095347
  0.010346747695920921
  0.07189316973140625
 -0.012658094161173916
 -0.10173386563905207
 -0.016997091931758357
  0.003054727340147334
  0.025347104928073128
 -0.005844452796460953
  ⋮
  0.00034418540329001455
 -0.00015840903795033773
 -6.849912934678813e-5
  0.00014981693924939754
 -3.002107124990775e-5
  0.00025918548486311005
 -8.855251967697809e-5
 -0.000222551093685101
  5.757400311573631e-5
  8.275763568502126e-6
 -6.635046832143476e-5
 -0.0012574575091876964

In [228]:
function f!(F,x)
    F = problem(x,A,beta,Omega,alpha,ε,θ,sigma,L)
end

f! (generic function with 1 method)

In [229]:
nlsolve(f!,x0)

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(f!), Float64}, Float64, 12})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at boot.jl:772
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  ...

In [ ]:
autodiff = :forward